### 多元线性回归中的梯度下降

 $-\eta\,\nabla J=-\eta \, ( \dfrac {\partial J}{\partial w_{1}},\dfrac {\partial J}{\partial w_{1}} ,\ldots ,\dfrac {\partial J}{\partial w_{n}},\dfrac {\partial J}{\partial b})$
 
 <img src="../img/2.png" width="500" align="left"/>

 <img src="../img/3.png" width="500" align="left"/>